In [1]:
!pwd

/Users/charlene/Library/Mobile Documents/com~apple~CloudDocs/NLPCourse/Lecture/jupyters_and_slides/2019-spring/assignments


## Assignments for Week-03

### 1. Re-code the titanic machine learning

###### 1. Random Choose Method to get optimal *k* and *b*
###### 2.Supervised Direction to get optimal *k* and *b*
###### 3.Gradient Descent to get optimal *k* and *b*

## 2. Answer following questions:


###### 1. Why we need machine learning methods instead of creating a complicated formula?


Ans:

###### 2.  Wha't's the disadvantages of `the 1st Random Choosen` methods in our course? 

Ans:

###### 3. Is the `2nd method supervised direction` better than 1st one?  What's the disadvantages of `the 2nd supversied directin` method? 

Ans:

###### 4. Why do we use `Derivative / Gredient` to fit a target function? 

Ans:

###### 5. In the words 'Gredient Descent', what's the `Gredient` and what's the `Descent`?

Ans:

###### 6. What's the advantages of `the 3rd gradient descent method` compared to the previous methods?

Ans:

###### 7. Using the simple words to describe: What's the machine leanring.

Ans:

## 3. Finish the search problem

Please using the search policy to implement an agent. This agent receives two input, one is @param start station and the other is @param destination. Your agent should give the optimal route based on Beijing Subway system. 

> Deadline: 2019-April-20

>Submit: Submit the source code and result to github. 



#### Dataflow: 

##### 1.	Get data from web page.

> a.	Get web page source from: https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81/408485

> b.	You may need @package requests page to get the response via url

> c.	You may need save the page source to file system.

> d.	The target of this step is get station information of all the subway lines;

> e.	You may need install @package beautiful soup  to get the url information, or just use > Regular Expression to get the url.  Our recommendation is that using the Regular Expression and BeautiflSoup both. 

> f.	You may need BFS to get all the related page url from one url. 
Question: Why do we use BFS to traverse web page (or someone said, build a web spider)?  Can DFS do this job? which is better? 

In [26]:
headers_strs = """
Connection: keep-alive
Cache-Control: no-cache
Accept: application/json, text/javascript, */*; q=0.01
X-Requested-With: XMLHttpRequest
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36
Accept-Encoding: gzip, deflate, br
Accept-Language: zh-CN,zh;q=0.9
Cookie: BAIDUID=343548C5C045A1498BAA1D382DC6D1CC:FG=1; BIDUPSID=343548C5C045A1498BAA1D382DC6D1CC; PSTM=1501732771; MCITY=-75%3A; pgv_pvi=9675362304; BDUSS=JDS3V-U0h2bUE4ckFFZjhreDhlUXN-fmlBRERuVWd-RFA0QWtyYTdYVUFMTjFjRVFBQUFBJCQAAAAAAAAAAAEAAAAlaCBMb2xkaXJvbnNpZGUAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACftVwAn7VcS3; pgv_si=s5658319872; H_PS_PSSID=28884_1440_28777_21086_28722_28558_28831_28585_26350_20718; Hm_lvt_55b574651fcae74b0a9f1cf9c8d7c93a=1555495877,1555498044,1555498060,1555546970; Hm_lpvt_55b574651fcae74b0a9f1cf9c8d7c93a=1555546970
""".split('\n')
headers = dict([(pair[:pair.find(':')], pair[pair.find(':')+1:].strip()) for pair in headers_strs if pair.strip()])

In [27]:
headers['User-Agent']

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'

In [36]:
import requests
from bs4 import BeautifulSoup 
start_url = "https://baike.baidu.com/item/北京地铁/408485"
start_content = requests.get(start_url, allow_redirects=False, headers=headers)
start_content.encoding = 'utf-8'
start_soup = BeautifulSoup(start_content.text)

In [51]:
start_table = start_soup.find('caption',text='北京地铁开通简表').parent


In [73]:
all_subway_url = dict([(a.string, 'https://baike.baidu.com'+a.attrs['href']) for a in start_table.select('a')])

In [137]:
import re
class SubwayLine():
    def __init__(self, soup):
        self.soup = soup
        self.stations, self.station_distaneces, self.bidirections = SubwayLine.parse_station_distance(soup)
    
    @staticmethod
    def parse_station_distance(soup):
        table = None
        for caption in soup.find_all('caption'):
            print(caption, caption.string)
            if caption.string is not None and re.match('\w+相邻站间距信息统计表', caption.string):
                table = caption.parent
                break
        if table is None:
            print("Error in parse station distance, cannot find table 相邻站间距信息统计表")
            return None,None,None
        
        stations = []
        distances = {}
        bidirections = {}
        s = None
        print('table:{}\n'.format(table))
        for tr in table.select('tr'):
            t = tr.contents
            print('t:{}\n'.format(t))
            if len(t)!=3:
                print('Warning: tr has unexpected numbers of children:{}'.format(tr))
                continue
            if not re.match('[\d|.]+[千米|米]', t[1].string):
                continue
            s = next(t[0].strings)
            print("s in loop:{}".format(s))
            stations.append(s.split('——')[0])
            distances[s] = float(t[1].string[:-2])*1000 if '千米' in t[1].string else float(t[1].string[:-1])
            bidirections[s] = t[1].string
               
#         s = next(tr.contents[0].strings)
        if s is None:
            print("out of loop is None")
        print(s.split('——'))
        stations.append(s.split('——')[1])
        return stations, distances, bidirections
                

In [138]:

current_response = requests.get(all_subway_url['北京地铁4号线'], headers=headers)
current_response.encoding = 'utf-8'
subways['北京地铁4号线'] = SubwayLine(BeautifulSoup(current_response.text))

<caption>北京地铁4号线、大兴线线路数据表</caption> 北京地铁4号线、大兴线线路数据表
<caption>北京地铁4号线、大兴线车站列表</caption> 北京地铁4号线、大兴线车站列表
<caption>4号线相邻站间距信息统计表</caption> 4号线相邻站间距信息统计表
table:<table class="four" log-set-param="table_view" width="658"><caption>4号线相邻站间距信息统计表</caption><tr><th>起始/终到车站</th><th>区间距离（米）</th><th>方向</th></tr><tr><th>安河桥北——北宫门</th><td align="center" valign="middle" width="198">1363</td><td align="center" valign="middle" width="198">上行/下行</td></tr><tr><th>北宫门——西苑</th><td align="center" valign="middle" width="198">1251</td><td align="center" valign="middle" width="198">上行/下行</td></tr><tr><th>西苑——圆明园</th><td align="center" valign="middle" width="198">1672</td><td align="center" valign="middle" width="198">上行/下行</td></tr><tr><th>圆明园——北京大学东门</th><td align="center" valign="middle" width="198">1295</td><td align="center" valign="middle" width="198">上行/下行</td></tr><tr><th>北京大学东门——中关村</th><td align="center" valign="middle" width="198">887</td><td align="center" valign="middle" width="198">上行/下行</td></tr><

AttributeError: 'NoneType' object has no attribute 'split'

In [133]:
subways = {}
for subway_name,subway_url in all_subway_url.items():
    current_response = requests.get(subway_url, headers=headers)
    current_response.encoding = 'utf-8'
    subways[subway_name] = SubwayLine(BeautifulSoup(current_response.text))
    

<caption>北京地铁1号线线路数据</caption> 北京地铁1号线线路数据
<caption>北京地铁1号线车站列表</caption> 北京地铁1号线车站列表
<caption>1号线相邻站间距信息统计表</caption> 1号线相邻站间距信息统计表
s in loop:苹果园——古城
s in loop:古城——八角游乐园
s in loop:八角游乐园——八宝山
s in loop:八宝山——玉泉路
s in loop:玉泉路——五棵松
s in loop:五棵松——万寿路
s in loop:万寿路——公主坟
s in loop:公主坟——军事博物馆
s in loop:军事博物馆——木樨地
s in loop:木樨地——南礼士路
s in loop:南礼士路——复兴门
s in loop:复兴门——西单
s in loop:西单——天安门西
s in loop:天安门西——天安门东
s in loop:天安门东——王府井
s in loop:王府井——东单
s in loop:东单——建国门
s in loop:建国门——永安里
s in loop:永安里——国贸
s in loop:国贸——大望路
s in loop:大望路——四惠
s in loop:四惠——四惠东
['四惠', '四惠东']
<caption>北京地铁1号线线路数据</caption> 北京地铁1号线线路数据
<caption>北京地铁1号线车站列表</caption> 北京地铁1号线车站列表
<caption>1号线相邻站间距信息统计表</caption> 1号线相邻站间距信息统计表
s in loop:苹果园——古城
s in loop:古城——八角游乐园
s in loop:八角游乐园——八宝山
s in loop:八宝山——玉泉路
s in loop:玉泉路——五棵松
s in loop:五棵松——万寿路
s in loop:万寿路——公主坟
s in loop:公主坟——军事博物馆
s in loop:军事博物馆——木樨地
s in loop:木樨地——南礼士路
s in loop:南礼士路——复兴门
s in loop:复兴门——西单
s in loop:西单——天安门西
s in loop:天安门西——天安门东
s in loop:天安门东——王府井
s i

UnboundLocalError: local variable 's' referenced before assignment

In [142]:
import sys
print(sys.executable)


/anaconda3/bin/python


In [140]:
user_paths

[]

In [99]:
subways['北京地铁1号线'].stations, subways['北京地铁1号线'].station_distaneces, subways['北京地铁1号线'].bidirections

(['苹果园',
  '古城',
  '八角游乐园',
  '八宝山',
  '玉泉路',
  '五棵松',
  '万寿路',
  '公主坟',
  '军事博物馆',
  '木樨地',
  '南礼士路',
  '复兴门',
  '西单',
  '天安门西',
  '天安门东',
  '王府井',
  '东单',
  '建国门',
  '永安里',
  '国贸',
  '大望路',
  '四惠',
  '四惠东'],
 {'苹果园——古城': 2606.0,
  '古城——八角游乐园': 1921.0,
  '八角游乐园——八宝山': 1953.0,
  '八宝山——玉泉路': 1479.0,
  '玉泉路——五棵松': 1810.0,
  '五棵松——万寿路': 1778.0,
  '万寿路——公主坟': 1313.0,
  '公主坟——军事博物馆': 1172.0,
  '军事博物馆——木樨地': 1166.0,
  '木樨地——南礼士路': 1291.0,
  '南礼士路——复兴门': 424.0,
  '复兴门——西单': 1590.0,
  '西单——天安门西': 1217.0,
  '天安门西——天安门东': 925.0,
  '天安门东——王府井': 852.0,
  '王府井——东单': 774.0,
  '东单——建国门': 1230.0,
  '建国门——永安里': 1377.0,
  '永安里——国贸': 790.0,
  '国贸——大望路': 1385.0,
  '大望路——四惠': 1673.0,
  '四惠——四惠东': 1714.0},
 {'苹果园——古城': '上行/下行',
  '古城——八角游乐园': '上行/下行',
  '八角游乐园——八宝山': '上行/下行',
  '八宝山——玉泉路': '上行/下行',
  '玉泉路——五棵松': '上行/下行',
  '五棵松——万寿路': '上行/下行',
  '万寿路——公主坟': '上行/下行',
  '公主坟——军事博物馆': '上行/下行',
  '军事博物馆——木樨地': '上行/下行',
  '木樨地——南礼士路': '上行/下行',
  '南礼士路——复兴门': '上行/下行',
  '复兴门——西单': '上行/下行',
  '西单——天安门西': '上行/下行',


##### 2.	Preprocessing data from page source.

> a.	Based on the page source gotten from url. You may need some more preprocessing of the page. 

> b.	the Regular Expression you may need to process the text information.

> c.	You may need @package networkx, @package matplotlib to visualize data. 

> d.	You should build a dictionary or graph which could represent the connection information of Beijing subway routes. 

> e.	You may need the defaultdict, set data structures to implement this procedure. 

##### 3. Build the search agent

> a.	Build the search agent based on the graph we build.

> b.	As much as you can to use the already implemented search agent. You just need to define the is_goal, get_successor, strategy three functions. 


##### 4.	Create different policies for transfer system.

> a.	Define different policies for transfer system. 

> b.	Such as Shortest Path Priority（路程最短优先）, Minimum Transfer Priority(最少换乘优先), Comprehensive Priority(综合优先)

> c.	Implement Continuous transfer. Based on the Agent you implemented, please add this feature: Besides the @param start and @param destination two stations, add some more stations, we called @param by_way, it means, our path should from the start and end, but also include the  @param by_way stations. 

e.g 
```
1. Input:  start=A,  destination=B, by_way=[C] 
    Output: [A, … .., C, …. B]
2. Input: start=A, destination=B, by_way=[C, D, E]
    Output: [A … C … E … D … B]  
    # based on your policy, the E station could be reached firstly. 
![image.png](attachment:image.png)
```

##### 5.	Test your result with commercial applications. 

将你的结果和高德地图或者百度地图进行比较，如果有不同，请分析原因
